# Swamp Angel Observations

In [1]:
# netcdf/numpy/xray/stats
import numpy as np
from datetime import datetime, timedelta
import pandas as pd
import xarray as xr
from scipy.stats.stats import pearsonr

# OS interaction
import sys, pickle, os

# import plotting
import seaborn as sns
import matplotlib
from matplotlib.pyplot import subplots
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.basemap import Basemap

# Offline Turbulence Package
import turbpy

# Customize
sns.set_style("whitegrid")
sns.set_context('paper')
%matplotlib inline

In [2]:
# --------------------------------------------------------------------------------------------------------------------
# Directory Lists
# Unix
if 'linux' in sys.platform:
    dirPre = '/home/lapok/gdrive/'
# Mac
elif 'darwin' in sys.platform:
    dirPre = '/Users/karllapo/gdrive/'

dirProj = dirPre + 'SnowHydrology/proj/ModTsfc/'
dirPrint = dirProj + 'Graphics'
dirData = dirProj + 'data'

# Original Swamp Angel observations
dirDataRaw = dirPre + 'GroundObs/SenatorBeck/Raw'

## Read and format the Senator Beck data

In [3]:
# --------------------------------------------------------------------------------------------
# data parser functions
def parser(x):
    return(pd.datetime.strptime(x, '%Y %j %H'))

def hourConverter(x):
    x = float(x)-100.
    if x >= 1000.:
        x = str(int(x/100))
    elif x < 1000. and x >= 0.:
        x = '0'+str(int(x/100))
    return x

os.chdir(dirData + '/SWA')
SWA = xr.open_dataset('met_insitu_swa_2005_2012.nc')
SWA.rename({'Psurf': 'Press',
            'Qair': 'QS',
            'SWdown': 'SWdwn',
            'LWdown': 'LWdwn',
            'Wind': 'WIND',
            'Snowf': 'precipSnow',
            }, inplace=True)
SWA['Tair'] = SWA['Tair'] - 273.15
print(SWA)

<xarray.Dataset>
Dimensions:     (time: 61368)
Coordinates:
  * time        (time) datetime64[ns] 2005-10-01T01:00:00 ...
Data variables:
    LWdwn       (time) float32 234.0 233.8 233.4 231.9 231.2 230.6 231.4 ...
    Press       (time) float32 66863.0 66863.0 66863.0 66863.0 66863.0 ...
    QS          (time) float32 0.0055 0.00543 0.00566 0.00509 0.00531 ...
    Rainf       (time) float32 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    precipSnow  (time) float32 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    SWdwn       (time) float32 0.0 0.0 0.0 0.0 0.0 0.0 11.1 186.4 454.3 ...
    Tair        (time) float32 1.35001 1.35001 1.95001 1.55002 1.45001 ...
    WIND        (time) float32 0.6 0.9 1.0 0.7 0.9 0.9 1.0 1.0 1.0 1.0 1.1 ...


In [4]:
# --------------------------------------------------------------------------------------------
# Read the Senator Beck data
os.chdir(dirDataRaw)
fields_to_keep = ['Hour',
                  'Year',
                  'DOY',
                  'UpWind_Uavg_MS',
                  'PyDwn_Unfilt_W',
                  'PyUp_Unfilt_W',
                  'Pyrgeom_W',
                  'Sno_IR_C',
                  'Soil_Surf_C',
                  'UpAir_Min_C',
                  'UpAir_Max_C',
                  'Up_RH',
                  'Sno_Height_M',
                  'Day_H2O_mm',
                  'Baro_inHg',
                 ]

SWAtxt = pd.read_csv('SASP_1hr.withHeader.txt',
                  converters={'Hour': hourConverter},
                  parse_dates={'time': ['Year', 'DOY', 'Hour']},
                  date_parser=parser,
                  index_col='time',
                  usecols=fields_to_keep,
                  header=0)

SWAtxt = xr.Dataset.from_dataframe(SWAtxt)

In [5]:
# --------------------------------------------------------------------------------------------
# Refine the Dataset
SWAtxt.rename({'Sno_IR_C': 'Tsrf1',
               'Soil_Surf_C': 'Tsrf2',
               'PyDwn_Unfilt_W': 'SWup',
               'PyUp_Unfilt_W': 'SWdwn',
               'Pyrgeom_W': 'LWdwn',
               'UpWind_Uavg_MS': 'WIND',
               'Day_H2O_mm': 'precip',
               'Sno_Height_M': 'snowDepth',
               'Baro_inHg': 'Press',
               'Up_RH': 'RH'}, inplace=True)


<xarray.Dataset>
Dimensions:      (time: 84299)
Coordinates:
  * time         (time) datetime64[ns] 2003-11-08T13:00:00 ...
Data variables:
    WIND         (time) float64 nan nan nan nan nan nan nan nan nan nan nan ...
    SWup         (time) float64 nan nan nan nan nan nan nan nan nan nan nan ...
    SWdwn        (time) float64 nan nan nan nan nan nan nan nan nan nan nan ...
    LWdwn        (time) float64 nan nan nan nan nan nan nan nan nan nan nan ...
    Tsrf1        (time) float64 nan nan nan nan nan nan nan nan nan nan nan ...
    snowDepth    (time) float64 0.27 0.27 0.27 0.28 0.29 0.29 0.28 0.28 0.29 ...
    precip       (time) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    Press        (time) float64 30.23 30.24 30.24 30.24 30.24 30.25 30.26 ...
    Tsrf2        (time) float64 nan nan nan nan nan nan nan nan nan nan nan ...
    UpAir_Min_C  (time) float64 nan nan nan nan nan nan nan nan nan nan nan ...
    UpAir_Max_C  (time) float64 nan nan nan nan nan nan nan n

### Handle other variables, including surface temperature

In [6]:
# Combine surface temperature time series into a single variable
# CSAS dataset splits up skin temperature into two separate fields: 'Snow_IR_C' and 'Soil_Surf_C'.
# These two different fields may be defined by snow depth. Using my own criteria below.
SWA['Tsrf'] = SWAtxt.Tsrf1.combine_first(SWAtxt.Tsrf2)
SWA['snowDepth'] = SWAtxt.snowDepth

# Take daily average, reindex to half-hourly time series, use in snow presence criteria
TsrfDaily = SWA.Tsrf.resample(how='mean', freq='d', dim='time', label='left')
TsrfDaily = TsrfDaily.reindex_like(SWA, method='ffill')

# bare ground when no snowdepth recorded or the daily surface temperature is above freezing
groundSurfTemp = SWA.Tsrf[(SWA.snowDepth < 0.01) | (TsrfDaily > 0.5)]

# snow covered ground when snow is observed and the surface temperature is below freezing
snowSurfTemp = SWA.Tsrf[(SWA.snowDepth > 0.01) & (TsrfDaily < 0.5)]

# Assign to support xarray.Dataset
SWA['groundTs'] = groundSurfTemp
SWA['snowTs'] = snowSurfTemp

# Create snow presence variable
snowPres = ((SWA.snowDepth > 0.01) & (TsrfDaily < 0.5))
SWA['SP'] = snowPres.astype(int)

# RH for overview plots
SWA['RH'] = SWAtxt['RH']

# Total precip
SWA['precip'] = SWA['precipSnow'] + SWA['Rainf']

# Upwelling shortwave
SWA['SWup'] = SWAtxt['SWup']

# Bulk Richardson
RiBulk,_,_ = turbpy.bulkRichardson(SWA.Tair + 273.15, SWA.Tsrf + 273.15, SWA.WIND, 6 - SWA.snowDepth)
SWA['RiBulk'] = RiBulk

//anaconda/lib/python3.5/site-packages/xarray/core/variable.py:1130: RuntimeWarning: invalid value encountered in less
  if not reflexive
//anaconda/lib/python3.5/site-packages/xarray/core/variable.py:1130: RuntimeWarning: invalid value encountered in greater
  if not reflexive
//anaconda/lib/python3.5/site-packages/xarray/core/variable.py:1130: RuntimeWarning: divide by zero encountered in true_divide
  if not reflexive


In [7]:
# -------------------------------------------------------------------------
# Write netcdf
os.chdir(dirData)
SWA.to_netcdf('SWA.ModTsfc.nc')
SWAtxt.to_netcdf('SWA_fromWebsite.ModTsfc.nc')
print(SWA)

<xarray.Dataset>
Dimensions:     (time: 61368)
Coordinates:
  * time        (time) datetime64[ns] 2005-10-01T01:00:00 ...
Data variables:
    LWdwn       (time) float32 234.0 233.8 233.4 231.9 231.2 230.6 231.4 ...
    Press       (time) float32 66863.0 66863.0 66863.0 66863.0 66863.0 ...
    QS          (time) float32 0.0055 0.00543 0.00566 0.00509 0.00531 ...
    Rainf       (time) float32 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    precipSnow  (time) float32 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    SWdwn       (time) float32 0.0 0.0 0.0 0.0 0.0 0.0 11.1 186.4 454.3 ...
    Tair        (time) float32 1.35001 1.35001 1.95001 1.55002 1.45001 ...
    WIND        (time) float32 0.6 0.9 1.0 0.7 0.9 0.9 1.0 1.0 1.0 1.0 1.1 ...
    Tsrf        (time) float64 0.115 0.07 -0.022 -0.085 -0.184 -0.223 0.473 ...
    snowDepth   (time) float64 0.191 0.155 0.167 0.163 0.166 0.154 0.12 nan ...
    groundTs    (time) float64 0.115 0.07 -0.022 -0.085 -0.184 -0.223 0.473 ...
    snow

In [8]:
# -------------------------------------------------------------------------
# Write daily averages
os.chdir(dirData)
SWA_daily = SWA.resample(how='mean', freq='d', dim='time')
SWA_daily.to_netcdf('SWA.ModTsfc.Daily.nc')
print(SWA_daily)

<xarray.Dataset>
Dimensions:     (time: 2558)
Coordinates:
  * time        (time) datetime64[ns] 2005-10-01 2005-10-02 2005-10-03 ...
Data variables:
    LWdwn       (time) float64 250.5 257.1 284.7 311.3 207.8 257.0 211.5 ...
    Press       (time) float64 6.686e+04 6.686e+04 6.686e+04 6.686e+04 ...
    QS          (time) float64 0.005839 0.005305 0.005965 0.006475 0.003927 ...
    Rainf       (time) float64 0.0 0.0 0.0 0.0001238 0.0 0.0 0.0 0.000119 ...
    precipSnow  (time) float64 0.0 1.221e-05 0.0 0.0 0.0 0.0 0.0 0.0 ...
    SWdwn       (time) float64 231.3 149.4 132.0 64.29 233.0 180.6 230.5 ...
    Tair        (time) float64 6.298 6.242 7.288 4.583 2.008 3.429 4.025 ...
    WIND        (time) float64 1.061 1.188 1.425 1.7 1.417 1.079 1.142 1.142 ...
    Tsrf        (time) float64 7.974 6.249 7.51 5.552 5.394 6.164 6.559 ...
    snowDepth   (time) float64 0.1583 0.1569 0.161 0.1801 0.1795 0.2644 ...
    groundTs    (time) float64 7.974 6.249 7.51 5.552 5.394 6.164 6.559 ...
    